In [2]:
import os
import pandas as pd

In [3]:
# loading the stocks and options file names
options_dir = "dataset/options/"
stocks_dir = "dataset/stocks/"

options_files = os.listdir(options_dir)
stocks_files = os.listdir(stocks_dir)
print (options_files)
print(stocks_files)

['2013-01-02options.csv', '2013-01-03options.csv', '2013-01-04options.csv', '2013-01-07options.csv', '2013-01-08options.csv', '2013-01-09options.csv', '2013-01-10options.csv', '2013-01-11options.csv', '2013-01-14options.csv', '2013-01-15options.csv', '2013-01-16options.csv', '2013-01-17options.csv', '2013-01-18options.csv', '2013-01-22options.csv', '2013-01-23options.csv', '2013-01-24options.csv', '2013-01-25options.csv', '2013-01-28options.csv', '2013-01-29options.csv', '2013-01-30options.csv', '2013-01-31options.csv', '2013-02-01options.csv', '2013-02-04options.csv', '2013-02-05options.csv', '2013-02-06options.csv', '2013-02-07options.csv', '2013-02-08options.csv', '2013-02-11options.csv', '2013-02-12options.csv', '2013-02-13options.csv']
['2013-01-02stocks.csv', '2013-01-03stocks.csv', '2013-01-04stocks.csv', '2013-01-07stocks.csv', '2013-01-08stocks.csv', '2013-01-09stocks.csv', '2013-01-10stocks.csv', '2013-01-11stocks.csv', '2013-01-14stocks.csv', '2013-01-15stocks.csv', '2013-01

In [4]:
(input_length, output_length) = (5, 1) # input time features and output time feautres
time_interval = 1 # time interval
offset = 0 # from where we have to load the code

In [5]:
# load the datasets
options_input = []
stocks_input = []
options_output = []
stocks_output = []

i = offset
for x in range(input_length):
    options_input.append(pd.read_csv(options_dir + options_files[i]))
    stocks_input.append(pd.read_csv(stocks_dir + stocks_files[i]))
    i += time_interval
for x in range(output_length):
    options_output.append(pd.read_csv(options_dir + options_files[i]))
    stocks_output.append(pd.read_csv(stocks_dir + stocks_files[i]))
    i += time_interval
print (options_output[0].tail())

                 contract underlying  expiration  type  strike style   bid  \
509814  ZX130622P00002500         ZX  2013-06-22   put     2.5     A  0.05   
509815  ZX130622C00005000         ZX  2013-06-22  call     5.0     A  0.00   
509816  ZX130622P00005000         ZX  2013-06-22   put     5.0     A  1.60   
509817  ZX130622C00007500         ZX  2013-06-22  call     7.5     A  0.00   
509818  ZX130622P00007500         ZX  2013-06-22   put     7.5     A  3.90   

        bid_size  ask  ask_size  volume  open_interest  quote_date   delta  \
509814       NaN  0.4       NaN       0              0  2013-01-09 -0.1925   
509815       NaN  0.3       NaN       0              0  2013-01-09  0.2188   
509816       NaN  2.0       NaN       0              0  2013-01-09 -0.7815   
509817       NaN  0.2       NaN       0              0  2013-01-09  0.1276   
509818       NaN  4.4       NaN       0              0  2013-01-09 -0.9454   

         gamma   theta    vega  implied_volatility  
509814  0

In [6]:
# reset the type of date
for i in range(input_length):
    options_input[i]['period'] = pd.to_datetime(options_input[i]['expiration']) - pd.to_datetime(options_input[i]['quote_date'])
    options_input[i]['period'] = options_input[i]['period'].dt.days
for i in range(output_length):
    options_output[i]['period'] = pd.to_datetime(options_output[i]['expiration']) - pd.to_datetime(options_output[i]['quote_date'])
    options_output[i]['period'] = options_output[i]['period'].dt.days

In [7]:
OPTION_TYPE = ['call', 'put']
STYLE = ['A', 'E']
OPTION_VALUES = ['strike', 'bid', 'ask', 'volume', 'open_interest', 'delta', 'gamma', 'theta', 'vega', 'implied_volatility', 'period']
STOCK_VALUES = ['open', 'high', 'low', 'close', 'volume']

In [8]:
def create_one_hot_label(values, value):
    t = [0 for i in range(len(values))]
    t[values.index(value)] = 1
    return t

def create_one_X(contract, underlying, option_type, strike, style):
    X = [float(strike)]
    if option_type == 'call':
        X.append(0)
        X.append(1)
    else:
        X.append(1)
        X.append(0)
    if style == 'A':
        X.append(0)
        X.append(1)
    else:
        X.append(1)
        X.append(0)
    
    for i in range(input_length):
        df = options_input[i]
        option = df[(df['contract'] == contract) & (df['underlying'] == underlying) & (df['type'] == option_type)]
        df = stocks_input[i]
        stock = df[df['symbol'] == underlying]
        if len(option) == 0 or len(stock) == 0:
            return []
        for value in STOCK_VALUES:
            X.append(float(stock[value]))
        for value in OPTION_VALUES:
            if value == 'strike': pass
            else:
                print(i,option[value], float(option[value]))
                X.append(float(option[value]))

    return X

def create_one_y(contract, underlying, option_type):
    y = []
    for i in range(output_length):
        df = options_output[i]
        option = df[(df['contract'] == contract) & (df['underlying'] == underlying) & (df['type'] == option_type)]
        if len(option) == 0:
            return []
        y.append(float(option['bid']))
        y.append(float(option['ask']))
    return y

In [33]:
X = []
y = []
database = 10000
last_index = 0
for index, row in options_input[0].iterrows():
    if len(X) >= database:
        break
    last_index = index
    if row['period'] >= input_length + output_length:
        X.append(create_one_X(row['contract'], row['underlying'], row['type'], row['strike'], row['style']))
        y.append(create_one_y(row['contract'], row['underlying'], row['type']))

In [10]:
# import multiprocessing
# import numpy
# from concurrent.futures import ThreadPoolExecutor
# from multiprocessing import Pool, cpu_count

# X = []
# y = []
# database = 100
# last_index = 0

# def process_row(row):
#     if len(X) >= database:
#         return
#     if row['period'] >= input_length + output_length:
#         tmp = create_one_X(row['contract'], row['underlying'], row['type'], row['strike'], row['style'])
#         print(tmp)
#         X.append(tmp)
#         y.append(create_one_y(row['contract'], row['underlying'], row['type']))

# with ThreadPoolExecutor() as executor:
#     executor.map(process_row, [row for index, row in options_input[0].iterrows()])

# num_processes = int(cpu_count() * 0.8)

# with Pool(processes=num_processes) as pool:
#     pool.map(process_row, [1])

In [34]:
X = []
y = []
database = 10000
last_index = 0
for index, row in options_input[0][last_index:].iterrows():
    if len(X) >= database:
        break
    last_index = index
    if row['period'] >= input_length + output_length:
        x_data = create_one_X(row['contract'], row['underlying'], row['type'], row['strike'], row['style'])
        y_data = create_one_y(row['contract'], row['underlying'], row['type'])
        if len(x_data) == 0 or len(y_data) == 0:
            continue
        X.append(x_data)
        y.append(y_data)

import numpy as np
np.array(X, dtype=np.float32)
np.array(y, dtype=np.float32)
np.save('train/input_data.npy', X)
np.save('train/output_data.npy', y)